# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException
import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.pipeline.steps import AutoMLStep
from azureml.data.dataset_factory import TabularDatasetFactory

In [2]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'automl-capston-1'

experiment=Experiment(ws, experiment_name)

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [3]:
example_data = 'https://raw.githubusercontent.com/Keshav-agrawal2829/Heart-data/main/heart_failure_clinical_records_dataset.csv'
# dataset = Dataset.Tabular.from_delimited_files(example_data)        
#Register Dataset in Workspace
dataset = TabularDatasetFactory().from_delimited_files(example_data)
key = "heart_diesease_data"
dataset = dataset.register(workspace=ws,
                            name=key,
                            description='')

df = dataset.to_pandas_dataframe()
df.describe()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
count,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.00000,299.000000,299.000000,299.00000,299.000000,299.00000
mean,60.833893,0.431438,581.839465,0.418060,38.083612,0.351171,263358.029264,1.39388,136.625418,0.648829,0.32107,130.260870,0.32107
std,11.894809,0.496107,970.287881,0.494067,11.834841,0.478136,97804.236869,1.03451,4.412477,0.478136,0.46767,77.614208,0.46767
min,40.000000,0.000000,23.000000,0.000000,14.000000,0.000000,25100.000000,0.50000,113.000000,0.000000,0.00000,4.000000,0.00000
25%,51.000000,0.000000,116.500000,0.000000,30.000000,0.000000,212500.000000,0.90000,134.000000,0.000000,0.00000,73.000000,0.00000
50%,60.000000,0.000000,250.000000,0.000000,38.000000,0.000000,262000.000000,1.10000,137.000000,1.000000,0.00000,115.000000,0.00000
75%,70.000000,1.000000,582.000000,1.000000,45.000000,1.000000,303500.000000,1.40000,140.000000,1.000000,1.00000,203.000000,1.00000
max,95.000000,1.000000,7861.000000,1.000000,80.000000,1.000000,850000.000000,9.40000,148.000000,1.000000,1.00000,285.000000,1.00000


# Compute


In [4]:
amlcompute_cluster_name = "notebook252189"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

    compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)

Found existing cluster, use it.


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [5]:
# TODO: Put your automl settings here
automl_settings = {"experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'AUC_weighted'}

# TODO: Put your automl config here
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="DEATH_EVENT",   
                             path = './project-pipeline',
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings)

In [6]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)
from azureml.widgets import RunDetails
RunDetails(remote_run).show()

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
automl-capston-1,AutoML_801f076f-8f9e-4370-8baf-fc4dd1357129,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [7]:
remote_run.wait_for_completion(show_output=True)

Experiment,Id,Type,Status,Details Page,Docs Page
automl-capston-1,AutoML_801f076f-8f9e-4370-8baf-fc4dd1357129,automl,Running,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Cross validation
STATUS:       DONE
DESCRIPTION:  In order to accurately evaluate the model(s) trained by AutoML, we leverage a dataset that the model is not trained on. Hence, if the user doesn't provide an explicit validation dataset, a part of the training dataset is used to achieve this. For smaller datasets (fewer than 20,000 samples), cross-validation is leveraged, else a single hold-out set is split from the training data to serve as the validation dataset. Hence, for your input data we leverage cross-validation with 10 folds, if the number of training samples are fewer than 1000, and 3 folds in all other cases.
              Learn mo

{'runId': 'AutoML_801f076f-8f9e-4370-8baf-fc4dd1357129',
 'target': 'notebook252189',
 'status': 'Completed',
 'startTimeUtc': '2024-02-09T07:14:27.757554Z',
 'endTimeUtc': '2024-02-09T07:22:39.464996Z',
 'services': {},
 'warnings': [{'source': 'JasmineService',
   'message': 'No scores improved over last 10 iterations, so experiment stopped early. This early stopping behavior can be disabled by setting enable_early_stopping = False in AutoMLConfig for notebook/python SDK runs.'}],
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'AUC_weighted',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': None,
  'target': 'notebook252189',
  'AMLSettingsJsonString': '{"path":null,"name":"automl-capston-1","subscription_id":"a0a76bad-11a1-4a2d-9887-97a29122c8ed","resource_group":"aml-quickstarts-252189","workspace_name":"quick-starts-ws-252189","region":"southcentralus","compute_target":"no

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [8]:
best_run, fitted_model = remote_run.get_output()
# Print the best run
print(best_run)
# Get all metrics of the best run
best_run_metrics = best_run.get_metrics()
 
# Print all metrics of the best run
for metric_name in best_run_metrics:
    metric = best_run_metrics[metric_name]
    print(metric_name, metric)

Package:azureml-automl-runtime, training version:1.52.0.post1, current version:1.51.0.post1
Package:azureml-core, training version:1.52.0, current version:1.51.0
Package:azureml-dataprep, training version:4.11.4, current version:4.10.8
Package:azureml-dataprep-rslex, training version:2.18.4, current version:2.17.12
Package:azureml-dataset-runtime, training version:1.52.0, current version:1.51.0
Package:azureml-defaults, training version:1.52.0, current version:1.51.0
Package:azureml-interpret, training version:1.52.0, current version:1.51.0
Package:azureml-mlflow, training version:1.52.0, current version:1.51.0
Package:azureml-pipeline-core, training version:1.52.0, current version:1.51.0
Package:azureml-responsibleai, training version:1.52.0, current version:1.51.0
Package:azureml-telemetry, training version:1.52.0, current version:1.51.0
Package:azureml-train-automl-client, training version:1.52.0, current version:1.51.0.post1
Package:azureml-train-automl-runtime, training version:1.

Run(Experiment: automl-capston-1,
Id: AutoML_801f076f-8f9e-4370-8baf-fc4dd1357129_43,
Type: azureml.scriptrun,
Status: Completed)
matthews_correlation 0.6431941339840718
AUC_micro 0.9254583168186024
log_loss 0.3922091891117527
precision_score_weighted 0.8558205013850573
f1_score_micro 0.8459770114942529
balanced_accuracy 0.8133972278357289
recall_score_macro 0.8133972278357289
recall_score_micro 0.8459770114942529
AUC_macro 0.9197910773585258
average_precision_score_weighted 0.9336780077598512
average_precision_score_micro 0.9292329522586144
weighted_accuracy 0.8668847013162064
recall_score_weighted 0.8459770114942529
accuracy 0.8459770114942529
norm_macro_recall 0.6267944556714579
average_precision_score_macro 0.9118877715898369
precision_score_macro 0.8311809514650408
AUC_weighted 0.919791077358526
f1_score_macro 0.8135841361454466
f1_score_weighted 0.8430504070326592
precision_score_micro 0.8459770114942529
accuracy_table aml://artifactId/ExperimentRun/dcid.AutoML_801f076f-8f9e-4370

In [10]:
#TODO: Save the best model
best_model = best_run.register_model(model_path='outputs/model.pkl', model_name='automl_best_model_capstone',
                        properties={'AUC_weighted': best_run_metrics['AUC_weighted']})
 
print(best_model)

Model(workspace=Workspace.create(name='quick-starts-ws-252189', subscription_id='a0a76bad-11a1-4a2d-9887-97a29122c8ed', resource_group='aml-quickstarts-252189'), name=automl_best_model_capstone, id=automl_best_model_capstone:1, version=1, tags={}, properties={'AUC_weighted': '0.919791077358526'})


## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [14]:
import os
os.getcwd()
from azureml.core import Model

TODO: In the cell below, send a request to the web service you deployed to test it.

In [13]:

# Download scoring file 
best_run.download_file('outputs/scoring_file_v_1_0_0.py', 'scoreScript.py')

# Download environment file
best_run.download_file('outputs/conda_env_v_1_0_0.yml', 'envFile.yml')

TODO: In the cell below, print the logs of the web service and delete the service

In [15]:
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig

inference_config = InferenceConfig(entry_script='scoreScript.py',
                                    environment=best_run.get_environment())

# deploy
from azureml.core.webservice import AciWebservice

deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1)
service = Model.deploy(ws, "myservice", [best_model], inference_config, deployment_config)
service.wait_for_deployment(show_output = True)
print(service.state)

print(service.scoring_uri)

print(service.swagger_uri)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2024-02-09 08:01:18+00:00 Creating Container Registry if not exists.
2024-02-09 08:01:19+00:00 Registering the environment.
2024-02-09 08:01:19+00:00 Use the existing image.
2024-02-09 08:01:20+00:00 Submitting deployment to compute.
2024-02-09 08:01:25+00:00 Checking the status of deployment myservice..
2024-02-09 08:03:39+00:00 Checking the status of inference endpoint myservice.
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy
http://078460ad-dc34-4630-8431-c04d008bf1f0.southcentralus.azurecontainer.io/score
http://078460ad-dc34-4630-8431-c04d008bf1f0.southcentralus.azurecontainer.io/swagger.json


In [16]:
import json

#Import test data
test_df = df.sample(5) # data is the pandas dataframe of the original data
label_df = test_df.pop('DEATH_EVENT')

test_sample = json.dumps({'data': test_df.to_dict(orient='records')})

print(test_sample)

{"data": [{"age": 42.0, "anaemia": 0, "creatinine_phosphokinase": 5209, "diabetes": 0, "ejection_fraction": 30, "high_blood_pressure": 0, "platelets": 226000.0, "serum_creatinine": 1.0, "serum_sodium": 140, "sex": 1, "smoking": 1, "time": 87}, {"age": 45.0, "anaemia": 0, "creatinine_phosphokinase": 7702, "diabetes": 1, "ejection_fraction": 25, "high_blood_pressure": 1, "platelets": 390000.0, "serum_creatinine": 1.0, "serum_sodium": 139, "sex": 1, "smoking": 0, "time": 60}, {"age": 80.0, "anaemia": 0, "creatinine_phosphokinase": 776, "diabetes": 1, "ejection_fraction": 38, "high_blood_pressure": 1, "platelets": 192000.0, "serum_creatinine": 1.3, "serum_sodium": 135, "sex": 0, "smoking": 0, "time": 130}, {"age": 52.0, "anaemia": 0, "creatinine_phosphokinase": 132, "diabetes": 0, "ejection_fraction": 30, "high_blood_pressure": 0, "platelets": 218000.0, "serum_creatinine": 0.7, "serum_sodium": 136, "sex": 1, "smoking": 1, "time": 112}, {"age": 90.0, "anaemia": 1, "creatinine_phosphokinase"

In [17]:
import requests # Used for http post request

# Set the content type
headers = {'Content-type': 'application/json'}


response = requests.post(service.scoring_uri, test_sample, headers=headers)

In [18]:
print(response.text)

"{\"result\": [0, 1, 1, 0, 1]}"


In [19]:
print(service.get_logs())

2024-02-09T08:03:24,395144600+00:00 - rsyslog/run 
2024-02-09T08:03:24,411972900+00:00 - gunicorn/run 
2024-02-09T08:03:24,418186300+00:00 | gunicorn/run | 
2024-02-09T08:03:24,422338900+00:00 | gunicorn/run | ###############################################
2024-02-09T08:03:24,428830400+00:00 | gunicorn/run | AzureML Container Runtime Information
2024-02-09T08:03:24,436351500+00:00 | gunicorn/run | ###############################################
2024-02-09T08:03:24,438136900+00:00 - nginx/run 
2024-02-09T08:03:24,440394900+00:00 | gunicorn/run | 
2024-02-09T08:03:24,452862700+00:00 | gunicorn/run | 
2024-02-09T08:03:24,472714100+00:00 | gunicorn/run | AzureML image information: openmpi4.1.0-ubuntu20.04, Materializaton Build:20230628.v2
2024-02-09T08:03:24,475918300+00:00 | gunicorn/run | 
2024-02-09T08:03:24,482737600+00:00 | gunicorn/run | 
2024-02-09T08:03:24,485019400+00:00 | gunicorn/run | PATH environment variable: /azureml-envs/azureml-automl/bin:/opt/miniconda/bin:/usr/local/sbi

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
